# Amazon SageMaker Processing でデータ並列処理

Amazon SageMaker Processing を使うと、任意のコンテナイメージやソースコードを使ってデータ処理を行うことができます。処理したいファイル数が多い場合、SageMaker Processing で複数のインスタンスにデータを分散させて並列処理するとデータ処理時間を短縮を狙えます。このノートブックでは、SageMaker Processing を使ってデータ並列処理する方法をご紹介します。

## 使用するサービス
- Amazon SageMaker
- Amazon ECR
- Amazon S3

## docker イメージビルドのための下準備
コンテナイメージビルドの際の容量不足を回避するために以下のセルを実行して docker 関連のファイルの保存場所を変更してください。以下のセルは、ノートブックインスタンス起動後1度だけ実行してください。インスタンスを再起動した際はディレクトリ構成が元に戻ってしまうため、再度実行してください。

In [ ]:
%%bash

sudo service docker stop
sudo mv /var/lib/docker /home/ec2-user/SageMaker/docker
sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker
sudo service docker start

## 設定
SageMaker セッションを作成し、設定を開始します。入出力データを保存するための S3 バケットとプレフィックスは、ノートブックインスタンス、Processing Job と同じリージョン内にある必要があります。

In [ ]:
import boto3
from datetime import datetime
from dateutil import tz
import os
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
JST = tz.gettz('Asia/Tokyo')

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/proc-pytorch-mnist'

role = sagemaker.get_execution_role()

project_name = 'sagemaker-processing-parallel'
user_name = 'demo'

## データの取得

このノートブックでは MNIST データセットを使用します。あとで画像のリサイズ処理をデータ並列処理します。

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
!tar -xvzf  mnist_png.tgz

SageMaker Processing は Amazon S3 に保存されたデータを入力データとして指定する仕組みになっているため、データを S3 にアップロードします。10000画像あるため、データのアップロードが完了するまでに 10分ほどかかるかもしれません。

In [ ]:
data_dir = 'mnist_png/testing'
inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## SageMaker Processing の準備

### コンテナイメージのビルドと Amazon ECR へのプッシュ

SageMaker Processing は、データ処理に使用するコンテナイメージとスクリプトを個別に指定することができます。スクリプトをコンテナイメージの中に入れて実行することももちろん可能ですが、実行環境は変えずにスクリプトだけ変えたい場合もよくあると思うので、このノートブックではコンテナイメージとスクリプトを独立して扱います。

In [ ]:
!mkdir -p docker/prepro

In [ ]:
%%writefile docker/prepro/requirements.txt
pandas==1.0.4
Pillow==9.0.1

In [ ]:
%%writefile docker/prepro/Dockerfile
FROM python:3.8-slim-buster
    
ENV PYTHONUNBUFFERED=TRUE
RUN apt-get update -y && apt-get install -y libexpat1
COPY requirements.txt .
RUN pip3 install --upgrade pip
RUN pip3 install -U --no-cache-dir -r requirements.txt

In [ ]:
def build_and_push_image(repo_name, docker_path, extra_accounts=[], tag = ':latest'):
    uri_suffix = 'amazonaws.com'
    repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, repo_name + tag)

    !docker build -t $repo_name $docker_path
    for a in extra_accounts:
        !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {a}.dkr.ecr.{region}.amazonaws.com
    !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
    !aws ecr create-repository --repository-name $repo_name
    !docker tag {repo_name + tag} $repository_uri
    !docker push $repository_uri
    return repository_uri
image_repository_uri = build_and_push_image(project_name + '-prepro-' + user_name, './docker/prepro')

## SageMaker Processing Job の実行

以下のセルを実行すると、用意しておいたスクリプト [code/prepro.py](code/prepro.py) が SageMaker Processing Job として実行されます。以下の設定で Processing Job を実行すると、ml.t3.medium 4 台でデータ並列処理が実行され、処理が完了するまで 1時間ほどかかります。以下のセルで `Processor` を作成する際のパラメタ `instance_count` に 1を設定すると インスタンス 1台での処理となり Job の完了までに 3時間弱かかるため、データ並列処理の効果が出ているといえます。

In [ ]:
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

job_name = project_name + '-' + user_name + '-' + timestamp
code_path = '/opt/ml/processing/input/code'
input_path = '/opt/ml/processing/input/data'
output_path = '/opt/ml/processing/output/data'
output_s3_path = f's3://{bucket}/{job_name}/output/result'

processor = Processor(
    image_uri=image_repository_uri,
    entrypoint=["python3", f"{code_path}/prepro.py"],
#     env={"ENV": "value"},
    role=role,
    instance_count=4,
    instance_type="ml.t3.medium"
)

SCRIPT_LOCATION = "code"

code_s3_path = sagemaker_session.upload_data(
    SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(project_name, user_name, "code", timestamp),
)
code_s3_path


processor.run(
    job_name=job_name,
#     code=code_s3_path,
    inputs=[
        ProcessingInput(
            input_name='code',
            source=code_s3_path,
            destination=code_path),
        ProcessingInput(
            input_name='data',
            source=inputs,
            destination=input_path,
            s3_data_distribution_type='ShardedByS3Key')],
     outputs=[
         ProcessingOutput(
             output_name='result',
             source=output_path,
             destination=output_s3_path,
             s3_upload_mode='Continuous')],
    arguments=['--code-path', code_path,
              '--input-data-path', input_path,
              '--output-data-path', output_path,
              '--scale', '2.0'],
    logs=False,
    wait=False
)

from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://s3.console.aws.amazon.com/s3/buckets/{bucket}?region={region}&prefix={job_name}/output/result/&showversions=false\" target=\"_blank\">検出結果の生データ (S3)</a>"))

## 補足

上記リンクをクリックすると、SageMaker Processing で処理された結果が保存された S3 パスにジャンプします。`algo-n.txt` というファイルが保存されていますが、このファイルには各インスタンスが処理した画像名が記録されています。ファイル名の `n` はインスタンスの ID を示します。このノートブックの例では 10000画像を入力画像として使用しており、4台のインスタンスでデータ並列処理を実行するとそれぞれのインスタンスに 2500画像ずつ分配されます。

prepro.py では指定された倍率で入力画像をリサイズしています。最後に SageMaker がそれらの画像を S3 にアップロードします。リサイズ処理が高速すぎてデータ並列処理の効果が分かりにくかったため、70 行目に 1秒の sleep を入れました。インスタンスを起動したり入力データを S3 からダウンロードしたりするのにも時間がかかるため、データの数や処理時間によってはデータ分散の効果があまりえられないこともあります。ファイルサイズが小さいデータが大量にある場合（今回のようなケース）、ある程度の単位でデータを zip などでまとめるとデータのダウンロード時間を短縮できる可能性があります。